In [1]:
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from datasets import load_dataset 
from PIL import Image
import os, random
import pandas as pd
import transformers as tr
import json
import shutil
import copy

c:\Users\nicco\OneDrive\Desktop\DHDK\2ndYear\courses\MLfH\ex5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# DO NOT RUN CELL!

"""lines = []
with open("hateful_memes/train.jsonl") as f:
    lines = f.read().splitlines()

lines_dict = [json.loads(line) for line in lines]

df = pd.DataFrame(lines_dict)
print(df.loc[[0]])"""

      id            img  label  \
0  42953  img/42953.png      0   

                                               text  
0  its their character not their color that matters  


In [12]:
# DO NOT RUN CELL!
'''
imgs_referenced = df["img"]
imgs_referenced.astype(str)

def select_ref_imgs(orig_loc):
    orig_loc = str(orig_loc)
    file_name = orig_loc[4:]
   
    try:
        shutil.move(f".\hateful_memes\{orig_loc}", f".\hateful_memes\imgs_present/{file_name}")
        print(f"file {file_name} moved")
    except FileNotFoundError:
        print(f"file {file_name} not found")
        pass
    except Exception as e:
        print(f"error:{e}")

def change_slash(string):
    string = string.replace("/",  "\ " )
    string = string.replace(" ", "")
    return string
    '''

# imgs_referenced = imgs_referenced.apply(change_slash)
# imgs_referenced.apply(select_ref_imgs)

print("files moved")

files moved


In [13]:
# DO NOT RUN CELL!

# for file_name in os.listdir("./hateful_memes/img"):
    # file_path = os.path.join("./hateful_memes/img", file_name)
    # os.remove(file_path)
    # print(f"{file_path} deleted.")
    # pass
print ("files deleted")

files deleted


In [ ]:
# DO NOT RUN CELL!

# shutil.rmtree("hateful_memes/img_old")

In [7]:
# DO NOT RUN CELL!

# os.rename("hateful_memes/imgs_present", "hateful_memes/img")

In [14]:
# DO NOT RUN CELL!

"""def check_imgs_exist(file_path):
    file_path = str(file_path)
    file_name = file_path[4:]
    if os.path.isfile(f"hateful_memes/{file_path}"):
        # print(f"{file_name} present")
        return True
    else:
        return False
    
reference_df = df
reference_df["img_exists"] = reference_df["img"].apply(check_imgs_exist)
reference_df = reference_df[reference_df["img_exists"]]
reference_df = reference_df.drop(columns=["img_exists"])
reference_df = reference_df.reset_index()
"""

files present


In [4]:
# DO NOT RUN CELL!

# print(reference_df.head(), len(reference_df))

   index     id            img  label  \
0      0  42953  img/42953.png      0   
1      2  13894  img/13894.png      0   
2      4  82403  img/82403.png      0   
3      5  16952  img/16952.png      0   
4      6  76932  img/76932.png      0   

                                                text  
0   its their character not their color that matters  
1                           putting bows on your pet  
2  everybody loves chocolate chip cookies, even h...  
3           go sports! do the thing! win the points!  
4     fine you're right. now can we fucking drop it?   6744


In [ ]:
# DO NOT RUN CELL!

# print(reference_df.query("id == '14507'"))

     index     id            img  label  \
882   1112  14507  img/14507.png      1   

                                                  text  
882  when your history teacher starts talking about...  


In [15]:
# DO NOT RUN CELL!

# reference_df = reference_df.astype({"index":int, "id":int, "img":str, "label":float, "text":str})
# data = reference_df

# data.to_csv("data.csv")

In [2]:
data = pd.read_csv("data.csv")
print (data.head())

   Unnamed: 0  index     id            img  label  \
0           0      0  42953  img/42953.png    0.0   
1           1      2  13894  img/13894.png    0.0   
2           2      4  82403  img/82403.png    0.0   
3           3      5  16952  img/16952.png    0.0   
4           4      6  76932  img/76932.png    0.0   

                                                text  
0   its their character not their color that matters  
1                           putting bows on your pet  
2  everybody loves chocolate chip cookies, even h...  
3           go sports! do the thing! win the points!  
4     fine you're right. now can we fucking drop it?  


In [3]:
class DatasetTxt (Dataset):
    def __init__(self, df, tokenizer):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer

        self.text= df["text"].values
        self.label = df["label"].values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        text = self.text[index]
        label = self.label[index]

        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True, return_token_type_ids=False,
            padding='longest', return_attention_mask=True, return_tensors='pt', truncation=True)

        return {"input_ids": encoding["input_ids"].flatten(), "attention_mask": encoding["attention_mask"].flatten(),
                "labels": t.tensor(label, dtype=t.long)}
        
class DatasetImg (Dataset):
    def __init__(self, df, img_dir):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])

        self.img= df["img"].values
        self.label = df["label"].astype(int).values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        img_pth = os.path.join(self.img_dir, self.img[index])
        img = Image.open(img_pth).convert("RGB")
        img_tensor = self.transform(img).unsqueeze(0)
        img_tensor = img_tensor.squeeze(1)
        label = self.label[index]

        return img_tensor, label



In [95]:
# splitting data 50%
train_data, rest = train_test_split(data, test_size= 0.7)

In [5]:
tokenizer = tr.BertTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english", clean_up_tokenization_spaces=True)

In [96]:
text_ds = DatasetTxt(train_data, tokenizer=tokenizer)

In [97]:
imag_ds = DatasetImg(train_data, img_dir="hateful_memes/")

In [98]:
d_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)

In [99]:
train_loader_txt = DataLoader(text_ds, batch_size=16, shuffle=True, collate_fn=d_collator)
train_loader_img = DataLoader(imag_ds, batch_size=16, shuffle=True)

In [104]:
class MonoLateFusModel(nn.Module):
    def __init__(self, text_model, img_model, classes):
        super().__init__()
        self.text_model = text_model

        in_features_img = img_model.fc.in_features
        img_model.fc = nn.Linear(in_features_img, 2)
        self.img_model = img_model


        # architecture
        #ToxicBERT/HateBERT input
        self.textfc = nn.Linear(classes, 24)
        #ResNet input
        self.imgfc = nn.Linear(classes, 24)
        self.mlp = nn.Sequential(nn.Linear(24, 16), nn.ReLU(), nn.Linear(16, 2))

    def forward(self, tokenized_text, image):
        #to be done
        cloned_txt_model =copy.deepcopy(self.text_model)
        cloned_img_model = copy.deepcopy(self.img_model)

        if "labels" in tokenized_text.keys(): #check if the input has labels (model in training)
        
            toktext = tokenized_text["input_ids"]
            mask = tokenized_text["attention_mask"]
            labels = tokenized_text["labels"]

            # txt_proc = self.textfc(self.text_model(tokenized_text))
            # img_proc = self.imgfc(self.img_model(image))

            logits_for_txt = cloned_txt_model(input_ids=toktext, attention_mask=mask, labels=labels).logits
        
        else: #otherwise, model in evaluation, labels not required
            toktext = tokenized_text["input_ids"]
            mask = tokenized_text["attention_mask"]

            logits_for_txt = cloned_txt_model(input_ids=toktext, attention_mask=mask).logits
        
        img_t = cloned_img_model(image)

        #print(f"HateBERT output: {logits_for_txt.shape}")
        #print(f"ResNet-18 output: {img_t.shape}")

        txt_proc = self.textfc(logits_for_txt)
        img_proc = self.imgfc(img_t)

        #print(f"text data after mlp: {txt_proc.shape}")
        #print(f"image data after mlp: {img_proc.shape}")

        txt_fft = t.fft.fft(txt_proc)
        img_fft = t.fft.fft(img_proc)
        #print(f"txt_fft shape: {txt_fft.shape}",f"img_fft shape: {img_fft.shape}")
        fusion = t.real(txt_fft * img_fft)
        #print(f"fusion shape: {fusion.shape}")
        logits = self.mlp(fusion)
        #print(f"logits shape: {logits.shape}")

        return logits
    
    def fine_tune_subpart(self, dataloader, input_type):
        model = self.text_model if input_type == "txt" else self.img_model
        model.train()


        tot_loss = 0
        correct_preds = 0
        #tot_val = 0

        if input_type == "txt":
            for name, param in model.named_parameters():
                if "classifier" not in name:
                    param.requires_grad = False
            optimizer_txt = optim.AdamW(model.parameters(), lr= 5e-5)
            scheduler = optim.lr_scheduler.StepLR(optimizer_txt, step_size=1, gamma=0.9)
            
        elif input_type == "img":
            for name, param in model.named_parameters():
                if "fc" not in name:
                    param.requires_grad = False
            criterion = nn.CrossEntropyLoss()
            optimizer_img = optim.Adam(model.fc.parameters(), lr=0.001)

        for e in range(5):
            if input_type == "txt":
                for batch in dataloader:
                    toktext = batch["input_ids"]
                    mask = batch["attention_mask"]
                    labels = batch["labels"]

                    outputs = model(input_ids=toktext, attention_mask=mask, labels=labels)

                    loss = outputs.loss
                    logits = outputs.logits

                    optimizer_txt.zero_grad()
                    loss.backward()
                    optimizer_txt.step()
                    scheduler.step()
                    tot_loss += loss.item()

                    pred = t.argmax(logits, -1)
                # print(f"LOSS: {loss.item()}, PRED: {pred}")
                    

            elif input_type == "img":
                for images, labels in dataloader:

                    images = images.squeeze(1)

                    optimizer_img.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs.float(), labels)
                    loss.backward()
                    optimizer_img.step()
                    tot_loss += loss.item()
                    
                    #tot_val += labels.size(0)
                    
                    _, pred = t.max(outputs.data, 1)
                # print(f"LOSS: {loss.item()}, PRED: {pred}")

            
            else:
                print ("input_type must be either 'txt' or 'img'")
                
            correct_preds = t.sum(pred == labels)
            print(f"batch {e+1}/5", 
                  f"loss: {tot_loss/(len(dataloader)*(e+1)):.4f}, accuracy: {100*(correct_preds/(len(dataloader))):.2f}")
        return f"loss: {tot_loss/(len(dataloader)*(e+1))}, accuracy: {100*(correct_preds/(len(dataloader))):.2f}"

In [101]:
model_txt = tr.AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english", num_labels = 2)
model_img = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

In [105]:
hmc_lf_model = MonoLateFusModel(text_model=model_txt, img_model=model_img, classes=2)

In [103]:
hmc_lf_model.fine_tune_subpart(train_loader_txt, "txt")

batch 1/5 loss: 0.7418, accuracy: 3.1496
batch 2/5 loss: 0.7384, accuracy: 3.9370
batch 3/5 loss: 0.7369, accuracy: 4.7244
batch 4/5 loss: 0.7363, accuracy: 4.7244
batch 5/5 loss: 0.7362, accuracy: 4.7244


'loss: 0.7361526332736954, accuracy: 4.7244'

In [106]:
hmc_lf_model.fine_tune_subpart(train_loader_img, "img")

batch 1/5 loss: 0.6794, accuracy: 3.94
batch 2/5 loss: 0.6663, accuracy: 4.72
batch 3/5 loss: 0.6613, accuracy: 3.94
batch 4/5 loss: 0.6519, accuracy: 3.94
batch 5/5 loss: 0.6466, accuracy: 3.94


'loss: 0.6465948087023938, accuracy: 3.94'

In [107]:
class dataset_hmc (Dataset):
    def __init__(self, df, tokenizer, img_path):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer
        self.img_dir = img_path
        self.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])

        self.text= df["text"].values
        self.label = df["label"].values
        self.img = df["img"].values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        
        text = self.text[index]
        label = self.label[index]

        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True, return_token_type_ids=False,
            padding='longest', return_attention_mask=True, return_tensors='pt', truncation=True)
        
        img_pth = os.path.join(self.img_dir, self.img[index])
        img = Image.open(img_pth).convert("RGB")
        img_tensor = self.transform(img).unsqueeze(0)
        img_tensor = img_tensor.squeeze(1)
        label = self.label[index]

        return {"input_ids": encoding["input_ids"].flatten(), 
                "attention_mask": encoding["attention_mask"].flatten(),
                "img_tensor": img_tensor,
                "labels": t.tensor(label, dtype=t.long),
            }

In [108]:
train_hmc, test = train_test_split(rest, test_size=0.5)

In [109]:
ds_hmc = dataset_hmc(df=train_hmc, tokenizer=tokenizer, img_path="hateful_memes/")

In [110]:
dataloader_hmc = DataLoader(ds_hmc, batch_size=16, shuffle=True, collate_fn=d_collator)

In [112]:
hmc_lf_model.img_model.train()
for name, param in hmc_lf_model.img_model.named_parameters():
    if "fc" not in name:
        param.requires_grad = False

hmc_lf_model.text_model.train()
for name, param in hmc_lf_model.text_model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False

for name, param in hmc_lf_model.named_parameters():
    if not param.requires_grad and param.grad is not None:
        print(f"Warning: {name} has a gradient despite being frozen.")
    #print(f"{name}: requires_grad={param.requires_grad}")

hmc_lf_model.train()


total_loss = 0
correct_preds = 0

loss_function = nn.CrossEntropyLoss()
optimizer_general = t.optim.AdamW(filter(lambda p: p.requires_grad, hmc_lf_model.parameters()), 
    lr=1e-5
)
# optimizer_img = optim.Adam(hmc_lf_model.img_model.fc.parameters(), lr=0.001)
# optimizer_txt = optim.AdamW(hmc_lf_model.text_model.parameters(), lr= 5e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer_txt, step_size=1, gamma=0.9)

for e in range(5):
    for batch in dataloader_hmc:
        
        text_data = {
        "input_ids": batch["input_ids"],
        "attention_mask": batch["attention_mask"],
        "labels": batch["labels"]
        }
        img = batch["img_tensor"]
        img_squeezed = img.squeeze(1)
        label = batch["labels"]
        # print(text_data)

        outputs = hmc_lf_model(text_data,img_squeezed)
        #print (outputs)
        loss = loss_function(outputs, label)

        optimizer_general.zero_grad()
        loss.backward()
        optimizer_general.step()
        #print(f"loss: {loss.item()}")
        total_loss += loss.item()
        _, pred = t.max(outputs.data, 1)

        correct_preds += t.sum(pred == label)

    print(f"batch {e+1}/5", 
    f"loss: {total_loss/(len(dataloader_hmc)*(e+1)):.4f}, accuracy: {(correct_preds.float()/len(dataloader_hmc)*(e+1)):.2f}")

batch 1/5 loss: 0.6340, accuracy: 10.62
batch 2/5 loss: 0.6332, accuracy: 42.69
batch 3/5 loss: 0.6295, accuracy: 96.53
batch 4/5 loss: 0.6269, accuracy: 172.35
batch 5/5 loss: 0.6259, accuracy: 269.76


In [113]:
validation_ds, remaining = train_test_split(test, test_size=0.6)

In [114]:
ds_for_val = dataset_hmc(df=validation_ds, tokenizer=tokenizer, img_path="hateful_memes/")

In [91]:
loader_eval = DataLoader(ds_for_val, batch_size=16, shuffle=True, collate_fn=d_collator)

In [116]:
hmc_lf_model.text_model.eval()
hmc_lf_model.img_model.eval()
hmc_lf_model.eval()

with t.no_grad():
    val_loss = 0
    val_correct = 0

    for batch in loader_eval:
        
        text_data_val = {
        "input_ids": batch["input_ids"],
        "attention_mask": batch["attention_mask"]
        }
        img_val = batch["img_tensor"]

        img_val = img_val.squeeze(1)
        
        label_val = batch["labels"]
        # print(text_data)
        val_outputs = hmc_lf_model(text_data_val,img_val)
        #print (outputs)
        loss_val = loss_function(val_outputs, label_val)

        val_loss += loss_val.item()
        _, val_pred = t.max(val_outputs.data, 1)

        # print(f"predictions shape: {val_pred.shape}", f"labels shape: {label_val.shape}")

        val_correct += t.sum(val_pred == label_val)

    print(f"loss: {val_loss/len(loader_eval):.4f}, accuracy: {val_correct.float()/len(loader_eval):.2f}")

loss: 0.6084, accuracy: 10.97
